In [926]:
import pandas as pd
import numpy as np

# data = pd.read_excel('../data/master.xlsx')
# data = pd.read_excel('../data/master.xlsx').sample(1000)
data = {'A': [3, 3, 2], 'B': [3, 3, 4], 'C': [7, 7, 7]}
data = pd.DataFrame(data)

minSup = 1
numDims = len(data.columns)
outputRec = pd.DataFrame({
    "Combination": [],
    "Count": []
})


In [927]:
data.columns

Index(['A', 'B', 'C'], dtype='object')

In [928]:
data.head()

,A,B,C
0,3,3,7
1,3,3,7
2,2,4,7


In [929]:
# unique_counts = data.nunique()
# col_dict = dict(zip(unique_counts.index, unique_counts.values))
# sorted_cols = sorted(col_dict, key=col_dict.get, reverse=True)
# data = data[sorted_cols]

In [930]:
data.head()

,A,B,C
0,3,3,7
1,3,3,7
2,2,4,7


In [931]:
# def Partition(input_df, dim, start_row, end_row, dataCount):
#     subset_df = input_df.iloc[start_row:end_row + 1, :]
#     sorted_df = subset_df.sort_values(by=input_df.columns[dim])
#     input_df.iloc[start_row:end_row + 1, :] = sorted_df.values
#     counts = sorted_df.iloc[:, dim].value_counts().sort_index().tolist()
#     dataCount.extend(counts)


In [932]:
import heapq
import os
import openpyxl

def Partition(file_name, dim, start_row, end_row, dataCount):
    workbook = openpyxl.load_workbook(file_name)
    worksheet = workbook.active
    tempFileNames = []

    line_number = 0
    pageSize = 2
    pageIndex = 0
    df = pd.DataFrame()
    for row in worksheet.iter_rows(min_row=start_row+2, max_row=end_row+2, values_only=True):
        if line_number >= pageSize:
            df.sort_values(by=df.columns[dim], inplace=True)
            tempFileName = "../data/temporary/" + str(pageIndex) + ".xlsx"
            df.to_excel(tempFileName, index=False, header=False)
            tempFileNames.append(tempFileName)
            line_number = 0
            pageIndex += 1
            df = pd.DataFrame()

        df = df.append(pd.DataFrame([row]), ignore_index=True)
        line_number += 1

    if line_number > 0:
        df.sort_values(by=df.columns[dim], inplace=True)
        tempFileName = "../data/temporary/" + str(pageIndex) + ".xlsx"
        df.to_excel(tempFileName, index=False, header=False)
        tempFileNames.append(tempFileName)
        pageIndex += 1

    min_heap = []

    workbooks = [openpyxl.load_workbook(tempFileName) for tempFileName in tempFileNames]
    file_handles = [workbook.active for workbook in workbooks]
    iterators = [file.iter_rows(values_only=True) for file in file_handles]
    current_row = start_row+2

    # Push the <value, pageid, row> of the first row of each file_handle to the min heap
    for i, iterator in enumerate(iterators):
        first_row = next(iterator)
        heapq.heappush(min_heap, (first_row[dim], i, first_row))

    # While the heap is not empty
    while len(min_heap) > 0:
        value, file_index, row = heapq.heappop(min_heap)

        for col_index, cell_value in enumerate(row):
            worksheet.cell(row=current_row, column=col_index + 1, value=cell_value)

        current_row += 1  
    
        dataCountLength = len(dataCount)
        if dataCountLength == 0:
            dataCount[value] = 1
        elif value in dataCount:
            dataCount[value] += 1
        else:
            dataCount[value] = 1

        # Read the next row from the file of index page_index if it is not empty
        try:
            next_row = next(iterators[file_index])
            heapq.heappush(min_heap, (next_row[dim], file_index, next_row))
        except StopIteration:
            # The file is empty, no more rows to read from it
            continue

    # Save the changes to the original workbook
    workbook.save(file_name)

    workbook.close()
    for workbook in workbooks:
        workbook.close()

     # Delete all the temporary files
    for tempFileName in tempFileNames:
        os.remove(tempFileName)


In [933]:
def BottomUpCube(input, dim, start_row, end_row, currentCombination, numDims, minSup):
    global outputRec
    for d in range(dim, numDims):
        dataCount = {}
        Partition(input, d, start_row, end_row, dataCount)

        k = start_row
        for value, c in dataCount.items():
            if c >= minSup:
                outputRec = pd.concat([pd.DataFrame(outputRec), pd.DataFrame([{
                    "Combination": currentCombination + [value], 
                    "Count": c
                    }])], ignore_index=True)

                BottomUpCube(input, d + 1, k, k + c - 1, currentCombination + [value], numDims, minSup)

            k += c    

In [934]:
# count = len(data)
# if count >= minSup:
#     outputRec = pd.concat([pd.DataFrame(outputRec), pd.DataFrame([{"Combination": [], "Count": count}])], ignore_index=True)

In [935]:
numDims = 3
minSup = 1
fileName = "../data/inputs/test.xlsx"
BottomUpCube(fileName, 0, 0, 3, [], numDims, minSup)

/tmp/ipykernel_8664/3678795572.py:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.DataFrame([row]), ignore_index=True)
/tmp/ipykernel_8664/3678795572.py:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.DataFrame([row]), ignore_index=True)
/tmp/ipykernel_8664/3678795572.py:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.DataFrame([row]), ignore_index=True)
/tmp/ipykernel_8664/3678795572.py:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.DataFrame([row]), ignore_index=True)
/tmp/ipykernel_8664/3678795572.py:24: FutureWarning: The frame.append method is deprecat

/tmp/ipykernel_8664/3678795572.py:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.DataFrame([row]), ignore_index=True)
/tmp/ipykernel_8664/3678795572.py:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.DataFrame([row]), ignore_index=True)
/tmp/ipykernel_8664/3678795572.py:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.DataFrame([row]), ignore_index=True)
/tmp/ipykernel_8664/3678795572.py:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.DataFrame([row]), ignore_index=True)
/tmp/ipykernel_8664/3678795572.py:24: FutureWarning: The frame.append method is deprecat

In [936]:

print(outputRec)

    Combination  Count
0           [2]    1.0
1        [2, 3]    1.0
2     [2, 3, 7]    1.0
3        [2, 7]    1.0
4           [9]    1.0
5        [9, 6]    1.0
6     [9, 6, 5]    1.0
7        [9, 5]    1.0
8          [11]    2.0
9       [11, 3]    2.0
10  [11, 3, -1]    1.0
11   [11, 3, 3]    1.0
12     [11, -1]    1.0
13      [11, 3]    1.0
14          [3]    3.0
15      [3, -1]    1.0
16       [3, 3]    1.0
17       [3, 7]    1.0
18          [6]    1.0
19       [6, 5]    1.0
20         [-1]    1.0
21          [3]    1.0
22          [5]    1.0
23          [7]    1.0


In [937]:
# data.head()